# Mixed Type Problem
This is currently the most versatile way to define problems. 
Let's say that we have the following minimization problem:

\begin{equation}
\begin{aligned}
& \underset{\mathbf x}{\text{min}}
& & y_1, y_2, y_3\\
& & & y_1 = x_1 + x_2 + x_3 \\
& & & y_2 = x_1 * x_2 * x_3  \\
& & & y_3 = x_1 * x_2 + x_3 \\
& \text{s.t.} & &  -2 \leq x_1 \leq 5 \\
& & &  -1 \leq  x_2 \leq 10 \\
& & &  -0 \leq x_3 \leq 3 \\
& & &  x_1 + x_2 + x_3 \leq 10 \\
& & &  \mathbf{x} \; \in S, \\
\end{aligned}
\end{equation}


Import the necessary classes.

variable_builder: Enables you to define multiple variables at once.

ScalarObjective: Enables you to define objectives **1** objective at a time.

VectorObjective: Enables you to define multiple objectives at once. This is useful if the source of the objective value evaluator returns multiple objectives at once.

ScalarConstraint: Enables you to define constraints **1** constraint at a time.

MOProblem: Takes as its input: 
1. A list of variables (returned by variable_builder)
2. A list of (one or more of) ScalarObjective and/or VectorObjective instances
3. A list of ScalarConstraint instances

And creates a multiobjective problem class that can be used by the rest of the desdeo methods (present in desdeo-mcdm and desdeo-emo)

In [1]:
from desdeo_problem.Objective import ScalarObjective, VectorObjective
from desdeo_problem.Problem import MOProblem
from desdeo_problem.Variable import variable_builder
from desdeo_problem.Constraint import ScalarConstraint
import numpy as np
from pprint import pprint

Define the variables. To do this, you need to give a list of variable names, list of the initial values for those variables, list of lower bounds, and a list of upper bounds.

In [2]:
var_names = ["a", "b", "c"]
initial_values = [1, 1, 1]
lower_bounds = [-2, -1, 0]
upper_bounds = [5, 10, 3]
variables = variable_builder(var_names, initial_values, lower_bounds, upper_bounds)
print("Type of \"variables\": ", type(variables))
print("Length of \"variables\": ", len(variables))
print("Type of the contents of \"variables\": ", type(variables[0]))

Type of "variables":  <class 'list'>
Length of "variables":  3
Type of the contents of "variables":  <class 'desdeo_problem.Variable.Variable'>


Define the evaluators for the objectives. These evaluators should be python functions that take in the decision variable values and give out the objective value/s. The arguments of these evaluators are 2-D Numpy arrays.

In [3]:
def obj1_2(x):
    y1 = x[:, 0] + x[:, 1] + x[:, 2]
    y2 = x[:, 0] * x[:, 1] * x[:, 2]
    return (y1, y2)


def obj3(x):
    y3 = x[:, 0] * x[:, 1] + x[:, 2]
    return y3

Define the objectives. For this, you need the names of the objectives, and the evaluators defined above. If an evaluator returns multiple objective values, use the `VectorObjective` class to define those objectives. If an evaluator returns objective values for only one objective, either `VectorObjective` or `ScalarObjective` can be used.

If using `VectorObjective`, names should be provided in a list.

Additionaly, bounds of the objective values can also be provided.

In [4]:
f1_2 = VectorObjective(["y1", "y2"], obj1_2)
f3 = ScalarObjective("f3", obj3)

Define the constraints. Constraint may depend on objective function as well (second argument to the lambda, notice the underscore). In that case, the objectives should not be defined inline, like above, but as their own function definitions. The constraint should be defined so, that when evaluated, it should return a positive value, if the constraint is adhered to, and a negative, if the constraint is breached.

In [5]:
# Args: name, number of variables, number of objectives, callable
cons1 = ScalarConstraint("c_1", 3, 3, lambda x, _: 10 - (x[:, 0] + x[:, 1] + x[:, 2]))

Now, we can create the problem object.
Provide objectives, variables and constraints in lists.

In [6]:
prob = MOProblem(objectives=[f1_2, f3], variables=variables, constraints=[cons1])

The objectives and constraint values can now be evaluated using the `<problem instance>.evaluate` method.

Decision variable data can be provided as a 2-d array or as a tuple/list of arrays.

The output is a tuple. The first element contains the objective values, the second element contains the constraint violation values.

**Note**: Input as list of lists is not supported

In [7]:
data = np.asarray([[1, -1, 0], [5, 5, 2]])
res= prob.evaluate(data)

In [8]:
print(res)

Evaluation Results Object 
Objective values are: 
[[ 0. 12. -1.]
 [ 0. 50. 27.]]
Constraint violation values are: 
[[10.]
 [-2.]]
Fitness values are: 
[[ 0. 12. -1.]
 [ 0. 50. 27.]]
Uncertainity values are: 
[[nan nan nan]
 [nan nan nan]]



In [9]:
print("The objective values for the given set of decision variables are: \n", res.objectives)
print("The constraint violation for the given set of decision variables are:\n", res.constraints)

The objective values for the given set of decision variables are: 
 [[ 0. 12. -1.]
 [ 0. 50. 27.]]
The constraint violation for the given set of decision variables are:
 [[10.]
 [-2.]]


In [10]:
res

EvaluationResults(objectives=array([[ 0., 12., -1.],
       [ 0., 50., 27.]]), fitness=array([[ 0., 12., -1.],
       [ 0., 50., 27.]]), constraints=array([[10.],
       [-2.]]), uncertainity=array([[nan, nan, nan],
       [nan, nan, nan]]))